In [43]:
import pickle as pkl
from WebQA.core.QA_Page import QA_Page
from WebQA.pages.Medhelp import MedhelpPage
from WebQA.pages.Healthtap import HealthtapPage
from WebQA.core.Post import Post, Question, Answer
from WebQA.core.User import User
import pandas as pd
import sys
import os

In [44]:
HOME = os.path.expanduser("~")
DATA = HOME + "/data_buffer/WebQA/"
# in this preprocessing, I chose to use this seperator to seperate
# different answers to the same question.
SEPERATOR = "@@@@"

import unicodedata
import string
import re


def unicode_to_ascii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )


# Lowercase, trim, and remove non-letter characters
def normalize_string(s):
    s = unicode_to_ascii(s.lower().strip())
    s = ' '.join(s.split())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

In [45]:
def turn_to_pairs(filename: str = None):
    def question_filter(answer):
        if answer.user and answer.user.isMedical:
             return answer.text
        return ""
    
    with open(DATA + filename, "rb") as f:
        pages = pd.read_pickle(f)
    questions = []
    answers = []
    for page in pages:
        if page and page.question != "":
            #page.print()
            medical_answers = [question_filter(answer) for answer in page.answers]
            if medical_answers:
                questions.append(normalize_string(page.question.text))
                answers.append([normalize_string(answer) \
                                        for answer in medical_answers])
            else:
                continue
                
    training_pairs = []
    for i in range(len(answers)):
        training_pairs = [questions[i]  + "\t" + answer for answer in answers[i] if answer != ""]
    return training_pairs

In [46]:
def create_tsv(filename: str = None, content: list = None):
    with open(filename, "w") as record_file:
        for item in content:
            record_file.write("%s\n" % item)

In [ ]:
training_pairsQA = turn_to_pairs("qa.pkl")
training_pairsHealth = turn_to_pairs("healthtap.pkl")

In [48]:
training_pairsHealth[:3]
training_pairsQA[:3]

[]

In [101]:

create_tsv('QA_dataset.tsv', training_pairsQA)
create_tsv('Health_dataset.tsv', training_pairsHealth)

In [70]:
s = """
        @@@@With these features I could offer a clinical diagnosis of crohn's disease with ho of anal fissure,weight loss and joint pains. Cheers.	Hi, my boyfriend of seven years has an undiagnosed chronic joint pain condition. He is 25 years old. To start off with a little bit of history, he has had this as long as he can remember. When he was young, people just told him it was âgrowing painsâ. I canât go into too much detail about other symptoms, but should say he was infected with meningitis around the age of 11-12 and was hospitalized for a while. His momâs side is relatively healthy. His dad was diagnosed with breast and testicular cancer in his 20s and was diagnosed with advanced prostate cancer a couple of years ago. His paternal uncle has Crohnâs disease and diabetes. His nephew (3 years old) has some sort of undiagnosed condition. When I first met him, his joint pain was only related to weather (changes in pressure/cold weather) and did not affect his life too much. It was isolated to his knee joints. Over the past few years, there have been various other health issues that have come up, and have become increasingly worse and more alarming as times goes by. A few years ago he was diagnosed with an anal fissure which has since healed. Following this, he lost about 40lbs in the matter of months and started to look more sickly and pale. He sweats excessively at night to the point where he wakes up and the sheets are soaked. He has frequent headaches and bouts of nausea and vomiting more than a person should. His lungs are also phlegm and wheezy. His joint pain is getting so bad that he is almost debilitated and is getting to the point where almost every day is a bad day. The pain is also spreading to his wrists and elbows. The joints are not warm and do not swell or turn red. He has seen a rheumatologist and had blood tests and x-rays which came out clean. His blood tests showed âmarkersâ for arthritis but nothing conclusive. They did not look into it further and assumed he was lying or was looking for painkillers. He is starting to think he is crazy because no one believes him and is really depressed because this is basically ruining his life. He can barely do day to day activities. Please, does anyone know what this could be?
"""

In [5]:
objects = []
with open(DATA + "qa.pkl", "rb") as f:
    pages = pd.read_pickle(f)

In [7]:
print(pages[:2])

[None, None]
